# Climate Model and Climate data workshop, hands-on session

This jupyter notebook is part of the climate model and climate data workshop. It showcases how to use climate model data in R. However, it is not a R course!

Use kernel Teaching (R SLAC 2021).

Here we will use the ncdf4 (https://search.r-project.org/CRAN/refmans/ncdf4/html/00Index.html) library.

## 1 Loading and inspecting data

In [1]:
#-----------------------------------------------------------------------------#
# Load libraries
#-----------------------------------------------------------------------------#
library(ncdf4)

In [2]:
# get working directory
WD <- getwd()

# define directory where data is
indir <- "/net/co2/c2sm-data/rlorenz/climate_model_data_workshop/"
# define output directory in case you want to save any data
outdir <- paste(WD, "/data/", sep="")
print(outdir)

[1] "/home/rlorenz/scripts/climate_model_data_workshop/data/"


In [3]:
#-----------------------------------------------------------------------------#
# Read data
#-----------------------------------------------------------------------------#
file <- paste(indir, "tas_Amon_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_201001-201412.nc", sep="")
ex.nc <- nc_open(file)
print(paste("The file has",ex.nc$nvars,"variables"))
print(ex.nc)

[1] "The file has 5 variables"
File /net/co2/c2sm-data/rlorenz/climate_model_data_workshop/tas_Amon_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_201001-201412.nc (NC_FORMAT_NETCDF4_CLASSIC):

     5 variables (excluding dimension variables):
        double time_bnds[bnds,time]   
        double lat_bnds[bnds,lat]   
        double lon_bnds[bnds,lon]   
        double height[]   
            units: m
            axis: Z
            positive: up
            long_name: height
            standard_name: height
        float tas[lon,lat,time]   
            standard_name: air_temperature
            long_name: Near-Surface Air Temperature
            comment: near-surface (usually, 2 meter) air temperature
            units: K
            cell_methods: area: time: mean
            cell_measures: area: areacella
            history: 2019-09-04T13:47:47Z altered by CMOR: Treated scalar dimension: 'height'. 2019-09-04T13:47:47Z altered by CMOR: replaced missing value flag (-9e+33) and corresponding da

In [4]:
# This illustrates how to read all the data from a variable (the 5th) if you do not know the variable name
v5 <- ex.nc$var[[5]]
data1 <- ncvar_get( ex.nc, v5 )	# by default, reads ALL the data

print(paste("Var 5 has name",v5$name,"and",v5$ndim, "dimensions.",
    "Dimensions are:"))
print(v5$varsize)

[1] "Var 5 has name tas and 3 dimensions. Dimensions are:"
[1] 192  96  60


In [5]:
# Read variable "tas" directly if variable name is known
tas <- ncvar_get(ex.nc, "tas")
print(dim(tas))

[1] 192  96  60


❓ **Questions**

1. How many years are covered in the dataset?
2. What is the data frequency?
3. How many variables does the dataset contain?

### 1.1 Exercise
Calculate the grid spacing in latitudinal and longitudinal direction.

In [6]:
# Read longitude and latitude dimension
lon <- ncvar_get(ex.nc, "lon")
lat <- ncvar_get(ex.nc, "lat")

lat_spacing <- lat[2] - lat[1] 
lon_spacing <- lon[2] - lon[1]

print(paste("The latitudinal spacing is", lat_spacing))
print(paste("The longitudinal spacing is", lon_spacing))

[1] "The latitudinal spacing is 1.84963755933913"
[1] "The longitudinal spacing is 1.875"


## 2. Subsetting data

In [7]:
#Read time dimension
time <- ncvar_get(ex.nc, "time")
time_units <- ncatt_get(ex.nc, "time", "units") #time unit, for how defined
# extract base date, third bit of time_units "days since 1950-01-01 00:00"
base_date <- as.character(lapply(strsplit(as.character(time_units$value),
                                          split = " "), "[", 3))
time_d <- as.Date(time, format = "%j",
                  origin = as.Date(base_date)) # define time as date
time.y <- format(time_d, "%Y") #extract years only from time_d

Y.Start <- time.y[1] # first year in timeseries
                     # -> can be used as info for output file
Y.End <- time.y[length(time.y)] # last year in timeseries

In [8]:
# print start and end years, convert to integers
print(Y.Start)
print(Y.End)
first_year <- strtoi(Y.Start, base = 0L)
last_year <- strtoi(Y.End, base = 0L)

[1] "2010"
[1] "2014"


### Subset data for a specific year and print the dataset

#### Hint
R's which method can be used to find indeces where a specific condition is met, see:

- https://stat.ethz.ch/R-manual/R-devel/library/base/html/which.html

check ncvar_get and what kind of parameters can be used:
- https://www.rdocumentation.org/packages/ncdf4/versions/1.21/topics/ncvar_get

In [9]:
# Select the time slice you are interested in (e.g., time = 2012)
time_index <- which(time.y == 2012)
print(time_index)

# Read the variable data for the selected time slice
variable_name <- "tas"
data <- ncvar_get(ex.nc, variable_name, start = c(1, 1, time_index[1]), count = c(-1, -1, length(time_index)))
print(dim(data))

 [1] 25 26 27 28 29 30 31 32 33 34 35 36
[1] 192  96  12


In [10]:
# this is an example how a netcdf can be saved from R
print("save monthly data to netcdf")

#define dimensions for output file
dim1 <- ncdim_def("longitude", "degrees_east", as.double(lon))
dim2 <- ncdim_def("latitude", "degrees_north", as.double(lat))
dimT <- ncdim_def("time", "months", time_index, unlim = FALSE)
missval <- -9999

# define the EMPTY netcdf variable "tas"
var1 <- ncvar_def("tas", "K", list(dim1, dim2, dimT), missval,
                  longname = "temperature")

# associate the netcdf variable with a netcdf file  
file.out <- paste(outdir, "T_2012.nc", sep = "")
print(file.out)
# in case the output directory does not exist yet, create it
if (!file.exists(outdir)){
   dir.create(file.path(outdir), showWarnings = FALSE)
}

# create the netcdf file
nc.ex = nc_create(file.out, list(var1))

# put data into file
ncvar_put(nc.ex, var1, data) # one could write a subset of the data using
                           # start = c(1, 1, 1),  count = c(nlon, nlat, t2))
ncatt_put(nc.ex , var1, '_FillValue', missval )

[1] "save monthly data to netcdf"
[1] "/home/rlorenz/scripts/climate_model_data_workshop/data/T_2012.nc"


#### Exercise 2.1

Calculate the average temperature for the year 2012 and print the result.

In [11]:
data_mean <- mean(data)
print(paste("The mean temperature for 2012 is ", round(data_mean, digits = 2),".", sep=""))

[1] "The mean temperature for 2012 is 278.95."


### Subset data for a specific region using latitude and longitude bounds

In [12]:
# read data but only for polygon
min_lon <-  5. 
max_lon <- 16. 
# latitude is -90 to 90
min_lat <- 45.
max_lat <- 55.

# find start indices for these longitudes and latitudes
lon_index <- which(lon > min_lon & lon < max_lon)
print(lon_index)

lat_index <- which(lat > min_lat & lat < max_lat)
print(lat_index)

[1] 4 5 6 7 8 9
[1] 73 74 75 76 77


In [13]:
# Read the variable data for the selected latitudes and longitudes
data_reg <- ncvar_get(ex.nc, variable_name, start = c(lon_index[1], lat_index[1], 1), count = c(length(lon_index), length(lat_index), -1))
print(data_reg[,,1])

# Close the NetCDF file
nc_close(ex.nc)

         [,1]     [,2]     [,3]     [,4]     [,5]
[1,] 274.1919 274.3423 276.3071 276.2251 276.5923
[2,] 273.6646 273.3013 274.8833 274.6450 273.9634
[3,] 273.9478 272.4243 273.6313 273.5200 272.8560
[4,] 274.2153 272.0884 272.8071 273.0962 272.6294
[5,] 274.4927 272.1860 272.4341 272.9106 272.8345
[6,] 275.4185 272.9810 272.2642 272.7095 273.1665


#### Exercise 2.2

Calculate and print the mean temperature for the selected region.

In [14]:
data_reg_mean <- mean(data_reg)
print(paste("The mean temperature for the selected region is ", round(data_reg_mean, digits = 2)," K.", sep=""))

[1] "The mean temperature for the selected region is 281.03 K."


#### You can also use packages like "raster" to work with netcdf data

In [15]:
# load necessary packages
library(raster)

# get working directory and load function eurocentric.r
getwd()
source("eurocentric.r")

Loading required package: sp



[1] "/home/rlorenz/scripts/climate_model_data_workshop"

In [16]:
nc_file <- paste(indir, "tas_Amon_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_201001-201412.nc", sep="")
nc_data <- brick(nc_file) + 0
print(nc_data)

class      : RasterBrick 
dimensions : 96, 192, 18432, 60  (nrow, ncol, ncell, nlayers)
resolution : 1.875, 1.864677  (x, y)
extent     : -0.9375, 359.0625, -89.50451, 89.50451  (xmin, xmax, ymin, ymax)
crs        : +proj=longlat +datum=WGS84 
source     : memory
names      : X2010.01.16, X2010.02.15, X2010.03.16, X2010.04.16, X2010.05.16, X2010.06.16, X2010.07.16, X2010.08.16, X2010.09.16, X2010.10.16, X2010.11.16, X2010.12.16, X2011.01.16, X2011.02.15, X2011.03.16, ... 
min values :    233.5532,    226.7332,    218.3309,    208.2510,    207.4695,    208.3213,    206.3522,    202.6599,    202.6488,    209.7989,    223.7553,    233.1983,    234.8197,    226.6509,    216.7993, ... 
max values :    308.7876,    307.1746,    307.7919,    309.8936,    311.8543,    318.3056,    315.9889,    316.0739,    311.2015,    309.7032,    308.0228,    308.1046,    307.2845,    307.4575,    308.0845, ... 



In [17]:
nc_data = convert.to.eurocentric(nc_data)
print(nc_data)

class      : RasterBrick 
dimensions : 96, 192, 18432, 60  (nrow, ncol, ncell, nlayers)
resolution : 1.875, 1.864677  (x, y)
extent     : -180.9375, 179.0625, -89.50451, 89.50451  (xmin, xmax, ymin, ymax)
crs        : +proj=longlat +datum=WGS84 
source     : memory
names      : X2010.01.16, X2010.02.15, X2010.03.16, X2010.04.16, X2010.05.16, X2010.06.16, X2010.07.16, X2010.08.16, X2010.09.16, X2010.10.16, X2010.11.16, X2010.12.16, X2011.01.16, X2011.02.15, X2011.03.16, ... 
min values :    233.5532,    226.7332,    218.3309,    208.2510,    207.4695,    208.3213,    206.3522,    202.6599,    202.6488,    209.7989,    223.7553,    233.1983,    234.8197,    226.6509,    216.7993, ... 
max values :    308.7876,    307.1746,    307.7919,    309.8936,    311.8543,    318.3056,    315.9889,    316.0739,    311.2015,    309.7032,    308.0228,    308.1046,    307.2845,    307.4575,    308.0845, ... 



❓ **Question**

What does the function convert.to.eurocentric do?

In [18]:
# Extract the time dimension
nc <- nc_open(nc_file)

time <- ncvar_get(nc, "time")
time_units <- ncatt_get(nc, "time", "units")$value
print(time_units)
nc_close(nc)

# extract base date, third bit of time_units "days since 1950-01-01 00:00"
base_date <- as.character(lapply(strsplit(as.character(time_units),
                                          split = " "), "[", 3))
print(base_date)
time_dates <- as.Date(time, format = "%j",
                  origin = as.Date(base_date)) # define time as date
print(time_dates)

[1] "days since 1850-1-1 00:00:00"
[1] "1850-1-1"
 [1] "2010-01-16" "2010-02-15" "2010-03-16" "2010-04-16" "2010-05-16"
 [6] "2010-06-16" "2010-07-16" "2010-08-16" "2010-09-16" "2010-10-16"
[11] "2010-11-16" "2010-12-16" "2011-01-16" "2011-02-15" "2011-03-16"
[16] "2011-04-16" "2011-05-16" "2011-06-16" "2011-07-16" "2011-08-16"
[21] "2011-09-16" "2011-10-16" "2011-11-16" "2011-12-16" "2012-01-16"
[26] "2012-02-15" "2012-03-16" "2012-04-16" "2012-05-16" "2012-06-16"
[31] "2012-07-16" "2012-08-16" "2012-09-16" "2012-10-16" "2012-11-16"
[36] "2012-12-16" "2013-01-16" "2013-02-15" "2013-03-16" "2013-04-16"
[41] "2013-05-16" "2013-06-16" "2013-07-16" "2013-08-16" "2013-09-16"
[46] "2013-10-16" "2013-11-16" "2013-12-16" "2014-01-16" "2014-02-15"
[51] "2014-03-16" "2014-04-16" "2014-05-16" "2014-06-16" "2014-07-16"
[56] "2014-08-16" "2014-09-16" "2014-10-16" "2014-11-16" "2014-12-16"


In [19]:
# add time as date to rasterbrick
raster_ts <- setZ(nc_data, time_dates)
print(raster_ts)

class      : RasterBrick 
dimensions : 96, 192, 18432, 60  (nrow, ncol, ncell, nlayers)
resolution : 1.875, 1.864677  (x, y)
extent     : -180.9375, 179.0625, -89.50451, 89.50451  (xmin, xmax, ymin, ymax)
crs        : +proj=longlat +datum=WGS84 
source     : memory
names      : X2010.01.16, X2010.02.15, X2010.03.16, X2010.04.16, X2010.05.16, X2010.06.16, X2010.07.16, X2010.08.16, X2010.09.16, X2010.10.16, X2010.11.16, X2010.12.16, X2011.01.16, X2011.02.15, X2011.03.16, ... 
min values :    233.5532,    226.7332,    218.3309,    208.2510,    207.4695,    208.3213,    206.3522,    202.6599,    202.6488,    209.7989,    223.7553,    233.1983,    234.8197,    226.6509,    216.7993, ... 
max values :    308.7876,    307.1746,    307.7919,    309.8936,    311.8543,    318.3056,    315.9889,    316.0739,    311.2015,    309.7032,    308.0228,    308.1046,    307.2845,    307.4575,    308.0845, ... 
time       : 2010-01-16, 2014-12-16 (min, max)

